# Parsing bloomberg articles

In [1]:
!pip3 install pyarrow fastparquet nltk spacy bs4 lxml
!python3 -m spacy download en_core_web_sm

/home/hmont/Documents/cours/INSA-LYON/5IF/data-engineering/dataengtp/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 19.2 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import pandas as pd
import string
from collections import Counter
import nltk
from nltk.corpus import words, stopwords
import os
import pyarrow as pa
import pyarrow.parquet as pq
import spacy
from tqdm import tqdm  # For progress tracking
from multiprocessing import Pool  # For parallel processing
import psutil  # For monitoring memory

import requests
from bs4 import BeautifulSoup
import tarfile

/home/hmont/Documents/cours/INSA-LYON/5IF/data-engineering/dataengtp/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
# Define the base directory containing the articles
data_dir = "../data/"
bloomberg_dir = "../data/bloomberg_data/"  # Directory containing split files
output_tar = "../data/bloomberg_data/20061020_20131126_bloomberg_news_combined.tar.gz"

# Combine split parts into the original tar.gz file
with open(output_tar, "wb") as outfile:
    for part in sorted(os.listdir(bloomberg_dir)):
        if part.startswith("20061020_20131126_bloomberg_news.tar.gz."):
            part_path = os.path.join(bloomberg_dir, part)
            with open(part_path, "rb") as infile:
                outfile.write(infile.read())

# Extract the tar.gz file
with tarfile.open(output_tar, "r:gz") as tar:
    tar.extractall(bloomberg_dir)

print("Extraction complete.")

Extraction complete.


In [ ]:
# Initialize a list to store article data
articles = []

# Iterate over all date folders
for date_folder in os.listdir(bloomberg_dir):
    date_path = os.path.join(bloomberg_dir, date_folder)
    if os.path.isdir(date_path):
        # Iterate over all articles in the folder
        for article_file in os.listdir(date_path):
            article_path = os.path.join(date_path, article_file)
            if os.path.isfile(article_path):
                with open(article_path, "r", encoding="utf-8") as f:
                    lines = f.readlines()
                    # Ensure the article has the expected structure
                    if len(lines) >= 4:
                        title = lines[0].strip("-- ").strip()
                        author = lines[1].strip("-- ").strip()
                        date = lines[2].strip("-- ").strip()
                        link = lines[3].strip("-- ").strip()
                        content = "".join(lines[4:]).strip()
                        
                        # Append the article's data to the list
                        articles.append({
                            "title": title,
                            "author": author,
                            "date": date,
                            "link": link,
                            "content": content
                        })

# Create a pandas DataFrame from the articles list
df = pd.DataFrame(articles)

# Display the first few rows of the DataFrame
print(df.head())

# Save the DataFrame to a CSV file for future use
df.to_csv("aggregated_bloomberg_articles.csv", index=False, encoding="utf-8")

# Save the DataFrame to a parquet file
df.to_parquet("aggregated_bloomberg_articles.parquet", index=False)

print("DataFrame successfully saved to both CSV and parquet formats.")

Empty DataFrame
Columns: []
Index: []
DataFrame successfully saved to both CSV and parquet formats.


In [4]:
# Load the DataFrame (if already saved to a CSV file)
# Uncomment and adjust if loading from a CSV
# df = pd.read_csv("aggregated_bloomberg_articles.csv", encoding="utf-8")

# Basic exploration and consistency checks
print("DataFrame Overview:")
print(df.info())  # General info about the DataFrame

print("\nMissing Values:")
print(df.isnull().sum())  # Check for missing values in each column

print("\nDuplicate Rows:")
duplicates = df[df.duplicated()]
print(duplicates)  # Display duplicate rows if any

print("\nUnique Values in Each Column:")
for column in df.columns:
    print(f"{column}: {df[column].nunique()} unique values")

# Check the range of dates in the dataset
if "date" in df.columns:
    print("\nDate Range:")
    try:
        df["date"] = pd.to_datetime(df["date"])  # Ensure the date column is in datetime format
        print(f"Start: {df['date'].min()}, End: {df['date'].max()}")
    except Exception as e:
        print(f"Error parsing dates: {e}")

# Display a single full row by index
index_to_display = 0  # Replace with the desired row index
if 0 <= index_to_display < len(df):
    print("\nFull Row Display:")
    pd.set_option("display.max_colwidth", None)  # Remove truncation for text columns
    print(df.iloc[index_to_display])
    pd.reset_option("display.max_colwidth")
else:
    print(f"Index {index_to_display} is out of range.")

# Check column value distributions
print("\nColumn Value Distributions:")
for column in df.columns:
    print(f"\nDistribution of '{column}':")
    print(df[column].value_counts().head(10))  # Show top 10 most frequent values

DataFrame Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
None

Missing Values:
Series([], dtype: float64)

Duplicate Rows:
Empty DataFrame
Columns: []
Index: []

Unique Values in Each Column:
Index 0 is out of range.

Column Value Distributions:


In [5]:
# Ensure required nltk data is downloaded
nltk.download("words")
nltk.download("stopwords")

# Load the English dictionary and stopwords
english_words = set(words.words())
stop_words = set(stopwords.words("english"))

# Load the DataFrame
df = pd.read_csv("aggregated_bloomberg_articles.csv")
# df = pd.read_parquet("aggregated_bloomberg_articles.parquet")

# Function to clean and tokenize text
def tokenize_and_filter(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation and tokenize
    tokens = text.translate(str.maketrans("", "", string.punctuation)).split()
    
    # Filter out stopwords and short tokens
    filtered_tokens = [
        token for token in tokens
        if token not in stop_words and len(token) > 2
    ]
    return filtered_tokens

# Extract and process the 'content' column
all_tokens = []
for content in df["content"].dropna():  # Ensure non-null content
    all_tokens.extend(tokenize_and_filter(content))

# Identify non-dictionary words
non_dictionary_words = [
    word for word in all_tokens if word not in english_words
]

# Count word frequencies
word_counts = Counter(non_dictionary_words)

# Get the most common non-dictionary words (likely company names)
most_common_words = word_counts.most_common(500)

# Convert to DataFrame for analysis
result_df = pd.DataFrame(most_common_words, columns=["Word", "Frequency"])

# Save the result to CSV for further analysis
result_df.to_csv("non_dictionary_word_frequencies.csv", index=False)

# Display the top results
print(result_df)

[nltk_data] Downloading package words to /home/hmont/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hmont/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


EmptyDataError: No columns to parse from file

In [ ]:

# Load the spaCy model with "parser" disabled but keeping the "tagger"
nlp = spacy.load("en_core_web_sm", disable=["parser"])

# Load the DataFrame
df = pd.read_csv("aggregated_bloomberg_articles.csv")

# Function to extract company names (organizations) from the text
def extract_company_names(texts):
    companies = []
    for doc in nlp.pipe(texts, batch_size=25):  # Smaller batches to reduce memory usage
        companies.extend([ent.text for ent in doc.ents if ent.label_ == "ORG"])
    return companies

# Monitor system memory usage
def check_memory():
    mem = psutil.virtual_memory()
    print(f"Memory Usage: {mem.percent}% used. Available: {mem.available // (1024 ** 2)} MB.")

# Split data for multiprocessing
def parallel_processing(content):
    with Pool() as pool:
        # Split content into chunks for parallel processing
        chunk_size = len(content) // pool._processes
        chunks = [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]
        # Process in parallel
        results = pool.map(extract_company_names, chunks)
    return results

# Prepare the content
contents = df["content"].dropna().tolist()

# Run parallel extraction with memory tracking
print("Extracting company names...")
check_memory()  # Initial memory check
company_lists = parallel_processing(contents)
check_memory()  # Post-processing memory check

# Flatten the list of lists
all_companies = [company for sublist in company_lists for company in sublist]

# Count the frequency of each company name
print("Counting frequencies...")
company_counts = Counter(all_companies)

# Convert to DataFrame for analysis
company_counts_df = (
    pd.DataFrame(company_counts.items(), columns=["Company", "Frequency"])
    .sort_values(by="Frequency", ascending=False)
)

# Save the result to CSV
company_counts_df.to_csv("company_name_frequencies.csv", index=False)

# Display the top results
print(company_counts_df.head(20))

Extracting company names...
Memory Usage: 54.2% used. Available: 7197 MB.


KeyboardInterrupt: 

In [ ]:
# URL of the Wikipedia page for S&P 500 companies
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Fetch the page
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Extract the tables
table = soup.find("table", {"id": "constituents"})
table2 = soup.find("table", {"id": "changes"})

# Parse the tables into DataFrames
current_df = pd.read_html(str(table))[0]
changes_df = pd.read_html(str(table2))[0]

# Extract 'Symbol' and 'Security' from the current constituents
current_companies = current_df[["Symbol", "Security"]].copy()

# Extract companies added and removed from changes
added_companies = changes_df[["Added"]]
removed_companies = changes_df[["Removed"]]

# Drop the top-level column header
added_companies.columns = added_companies.columns.droplevel(0)
removed_companies.columns = removed_companies.columns.droplevel(0)

# Rename Ticker to Symbol
added_companies = added_companies.rename(columns={"Ticker": "Symbol"})
removed_companies = removed_companies.rename(columns={"Ticker": "Symbol"})

# Merge all DataFrames into one
all_companies = pd.concat([current_companies, added_companies, removed_companies], ignore_index=True).dropna()

# Remove duplicates if needed
all_companies = all_companies.drop_duplicates()

# Display the merged DataFrame
print(all_companies.info())
print(all_companies.head())

# Save the DataFrame to a CSV file
all_companies.to_csv("sp500_companies.csv", index=False)

/tmp/ipykernel_18502/277945778.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  current_df = pd.read_html(str(table))[0]
/tmp/ipykernel_18502/277945778.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  changes_df = pd.read_html(str(table2))[0]


<class 'pandas.core.frame.DataFrame'>
Index: 907 entries, 0 to 1210
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Symbol    907 non-null    object
 1   Security  907 non-null    object
dtypes: object(2)
memory usage: 21.3+ KB
None
  Symbol             Security
0    MMM                   3M
1    AOS          A. O. Smith
2    ABT  Abbott Laboratories
3   ABBV               AbbVie
4    ACN            Accenture


In [ ]:

# Ensure required nltk data is downloaded
nltk.download("words")
nltk.download("stopwords")

# Load the English dictionary and stopwords
from nltk.corpus import words, stopwords

english_words = set(words.words())
stop_words = set(stopwords.words("english"))

# Load the company data from the CSV
companies_df = pd.read_csv("sp500_companies.csv")

# Create a list of company names and their corresponding symbols (tickers)
companies = companies_df[["Symbol", "Security"]].dropna()
company_names = companies["Security"].str.lower().tolist()  # Convert to lowercase for case-insensitive matching
company_symbols = companies["Symbol"].str.lower().tolist()

# Load the articles DataFrame
df = pd.read_csv("aggregated_bloomberg_articles.csv")

# Function to clean and tokenize text
def tokenize_and_filter(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation and tokenize
    tokens = text.translate(str.maketrans("", "", string.punctuation)).split()
    
    # Filter out stopwords and short tokens
    filtered_tokens = [
        token for token in tokens
        if token not in stop_words and len(token) > 2
    ]
    return filtered_tokens

# Function to count company names in article text
def count_companies_in_text(text, company_names, company_symbols):
    tokens = tokenize_and_filter(text)
    company_count = Counter()

    # Count occurrences of company names and symbols in the tokens
    for token in tokens:
        if token in company_names:
            company_index = company_names.index(token)
            symbol = company_symbols[company_index]
            company_count[symbol] += 1

        # Also check for company symbols
        if token in company_symbols:
            company_count[token] += 1

    return company_count

# Dictionary to hold counts for all companies
company_frequency = Counter()

# Initialize the counter for articles
total_articles = len(df["content"].dropna())
processed_articles = 0

# Iterate through the articles and count company mentions
for content in df["content"].dropna():  # Ensure non-null content
    company_count = count_companies_in_text(content, company_names, company_symbols)
    company_frequency.update(company_count)
    
    # Update processed article count
    processed_articles += 1
    
    # Print progress every 100 articles
    if processed_articles % 100 == 0:
        print(f"Processed {processed_articles} / {total_articles} articles...")

# Convert the results to a DataFrame
result_df = pd.DataFrame(company_frequency.items(), columns=["Company", "Frequency"]).sort_values(by="Frequency", ascending=False)

# Save the result to CSV for further analysis
result_df.to_csv("company_mentions_frequencies.csv", index=False)

# Display the top results
print(result_df.head(20))

[nltk_data] Downloading package words to /home/hmont/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hmont/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processed 100 / 446810 articles...
Processed 200 / 446810 articles...
Processed 300 / 446810 articles...
Processed 400 / 446810 articles...
Processed 500 / 446810 articles...
Processed 600 / 446810 articles...
Processed 700 / 446810 articles...
Processed 800 / 446810 articles...
Processed 900 / 446810 articles...
Processed 1000 / 446810 articles...
Processed 1100 / 446810 articles...
Processed 1200 / 446810 articles...
Processed 1300 / 446810 articles...
Processed 1400 / 446810 articles...
Processed 1500 / 446810 articles...
Processed 1600 / 446810 articles...
Processed 1700 / 446810 articles...
Processed 1800 / 446810 articles...
Processed 1900 / 446810 articles...
Processed 2000 / 446810 articles...
Processed 2100 / 446810 articles...
Processed 2200 / 446810 articles...
Processed 2300 / 446810 articles...
Processed 2400 / 446810 articles...
Processed 2500 / 446810 articles...
Processed 2600 / 446810 articles...
Processed 2700 / 446810 articles...
Processed 2800 / 446810 articles...
P

In [ ]:
# Ensure required nltk data is downloaded
nltk.download("words")
nltk.download("stopwords")

# Load the English dictionary and stopwords
from nltk.corpus import words, stopwords

english_words = set(words.words())
stop_words = set(stopwords.words("english"))

# Load the company data from the CSV
companies_df = pd.read_csv("sp500_companies.csv")

# Create a list of company names
companies = companies_df["Security"].dropna()
company_names = companies.str.lower().tolist()  # Convert to lowercase for case-insensitive matching

# Load the articles DataFrame
df = pd.read_csv("aggregated_bloomberg_articles.csv")

# Function to clean and tokenize text
def tokenize_and_filter(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation and tokenize
    tokens = text.translate(str.maketrans("", "", string.punctuation)).split()
    
    # Filter out stopwords and short tokens
    filtered_tokens = [
        token for token in tokens
        if token not in stop_words and len(token) > 2
    ]
    return filtered_tokens

# Function to count company names in article text
def count_companies_in_text(text, company_names):
    tokens = tokenize_and_filter(text)
    company_count = Counter()

    # Count occurrences of company names in the tokens
    for token in tokens:
        if token in company_names:
            company_count[token] += 1

    return company_count

# Dictionary to hold counts for all companies
company_frequency = Counter()

# Initialize the counter for articles
total_articles = len(df["content"].dropna())
processed_articles = 0

# Iterate through the articles and count company mentions
for content in df["content"].dropna():  # Ensure non-null content
    company_count = count_companies_in_text(content, company_names)
    company_frequency.update(company_count)
    
    # Update processed article count
    processed_articles += 1
    
    # Print progress every 100 articles
    if processed_articles % 100 == 0:
        print(f"Processed {processed_articles} / {total_articles} articles...")

# Convert the results to a DataFrame
result_df = pd.DataFrame(company_frequency.items(), columns=["Company", "Frequency"]).sort_values(by="Frequency", ascending=False)

# Save the result to CSV for further analysis
result_df.to_csv("company_mentions_frequencies.csv", index=False)

# Display the top results
print(result_df.head(20))

[nltk_data] Downloading package words to /home/hmont/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hmont/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processed 100 / 446810 articles...
Processed 200 / 446810 articles...
Processed 300 / 446810 articles...
Processed 400 / 446810 articles...
Processed 500 / 446810 articles...
Processed 600 / 446810 articles...
Processed 700 / 446810 articles...
Processed 800 / 446810 articles...
Processed 900 / 446810 articles...
Processed 1000 / 446810 articles...
Processed 1100 / 446810 articles...
Processed 1200 / 446810 articles...
Processed 1300 / 446810 articles...
Processed 1400 / 446810 articles...
Processed 1500 / 446810 articles...
Processed 1600 / 446810 articles...
Processed 1700 / 446810 articles...
Processed 1800 / 446810 articles...
Processed 1900 / 446810 articles...
Processed 2000 / 446810 articles...
Processed 2100 / 446810 articles...
Processed 2200 / 446810 articles...
Processed 2300 / 446810 articles...
Processed 2400 / 446810 articles...
Processed 2500 / 446810 articles...
Processed 2600 / 446810 articles...
Processed 2700 / 446810 articles...
Processed 2800 / 446810 articles...
P